DBSCAN
===============
_Density-based spatial clustering of applications with noise_
***

TODO: Explanantion of DBSCAN

Clustering Of _Waking-up_ and _Sleep_ timings
=================
*****************
The following algorithm below will cluster sleep periods using the [DBSCAN algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) from the [scikit-learn library](http://scikit-learn.org/)

Input data requirements
---------------------
****
The data input is expected to follow the following format:

<table>
<th>TODO</th>
</table>

In [ ]:
%matplotlib inline
###########################################IMPRORTS#########################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pi
from sklearn.cluster import DBSCAN
###########################################IMPRORTS#########################################


# To return value in mins / total mins in one day
def to_mins(x):
    x = pd.Timestamp(x)
    year = x.year
    month =  x.month
    day = x.day
    return (x.value - pd.Timestamp(str(year)+'-'+str(month)+'-'+str(day)).value) / (60 * (10**9))

# Helper method to convert values to radian
def convert_to_radian(x):
    return ((x / (24*60)) * 2 * pi)

